In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [2]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Якимов"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

In [3]:
# Читаем данные из файла
train_data = pd.read_csv("datasets/rus_stocks_funds_train.csv")

In [4]:
train_data.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,7.210000e+09,2.870000e+09,110809.6875,3190.000000,9.870000,0.27
1,1.060000e+10,1.130000e+09,318000.0000,28230.000000,13.910000,0.30
2,2.249000e+10,1.027000e+10,329570.0000,61044.948718,4.080000,0.25
3,4.995200e+08,1.964200e+08,59380.0000,81.000000,2.910000,1.31
4,9.520000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.35


In [5]:
#pairplot(train_data)

In [6]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

66


In [7]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, test_size=val_size, random_state=random_state)
print(len(train), len(val))

263 66


In [8]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
y_column = 'Debt/Assets'
x_columns = [e for e in train_data.columns if e != y_column]

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), x_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Total issued ordinary shares',
                                  'Number of shares available for sale',
                                  'Number of employees',
                                  'Number of shareholders', 'Price/Profit'])])

In [9]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

In [10]:
# Устанавливаем названия столбцов
column_names = list(x_columns) + [y_column]
sc_train.columns = column_names
sc_val.columns = column_names

In [11]:
sc_train

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,0.005863,0.003861,0.231278,0.692601,0.010846,0.21
1,0.015666,0.025894,0.231278,0.149645,0.012778,0.29
2,0.000278,0.025894,0.231278,0.149645,0.063121,0.35
3,0.000024,0.025894,0.231278,0.149645,0.030429,0.36
4,0.000019,0.025894,0.231278,0.149645,0.075086,0.10
...,...,...,...,...,...,...
258,0.000065,0.025894,0.231278,0.149645,0.041802,0.83
259,0.000275,0.000597,0.231278,0.000221,0.008694,0.40
260,0.002149,0.025894,0.231278,0.149645,0.017827,0.11
261,0.000009,0.000052,0.009890,0.149645,0.274392,0.16


In [12]:
# Отберем необходимые параметры
x_train = sc_train[x_columns]
x_val = sc_val[x_columns]

y_train = (sc_train[y_column].values).flatten()
y_val = (sc_val[y_column].values).flatten()

In [13]:
model = DecisionTreeRegressor(criterion='absolute_error')

In [14]:
# Обучим модель
model.fit(x_train, y_train)

DecisionTreeRegressor(criterion='absolute_error')

In [15]:
# Проверим работу обученной модели на валидационной выборке
pred_val = model.predict(x_val)
mse = mean_squared_error(y_val, pred_val)
print(mse)

0.029277203781407187


In [16]:
pred_val

array([0.62, 0.81, 0.34, 0.11, 0.11, 0.17, 0.27, 0.24, 1.41, 0.1 , 0.38,
       1.41, 0.28, 0.27, 0.35, 0.13, 0.53, 0.21, 0.57, 0.5 , 0.21, 0.64,
       0.08, 0.18, 0.04, 0.27, 0.62, 0.16, 0.54, 0.84, 0.19, 0.21, 0.54,
       0.  , 0.01, 0.28, 0.24, 0.07, 0.21, 0.04, 0.34, 0.54, 0.12, 0.24,
       0.21, 0.48, 0.36, 0.33, 0.84, 0.3 , 0.54, 0.29, 0.1 , 0.48, 0.53,
       0.62, 0.04, 0.41, 0.54, 0.  , 0.48, 0.54, 1.3 , 1.41, 0.51, 0.22])

In [17]:
test = pd.read_csv("datasets/rus_stocks_funds_test.csv")

In [18]:
test[y_column] = 0.0

In [19]:
test

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.0
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.0
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.0
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.0
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.0
...,...,...,...,...,...,...
77,1.528600e+08,1.309960e+11,110809.6875,61044.948718,6.270000,0.0
78,1.295000e+08,2.634000e+07,110809.6875,25.000000,15.525191,0.0
79,2.246000e+10,0.000000e+00,281550.0000,61044.948718,6.340000,0.0
80,1.295000e+08,3.371000e+07,110809.6875,25.000000,6.320000,0.0


In [20]:
sc_test = pd.DataFrame(ct.transform(test))
sc_test.columns = column_names

In [21]:
x_test = sc_test[x_columns]

In [22]:
test[y_column] = model.predict(x_test)

In [23]:
test.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.200
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.120
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.270
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.480
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.705


In [24]:
test.to_csv('task5_results/Якимов - Задание 15.csv', index=False)